<h2>Segmenting and Clustering Neighborhoods in Toronto<h2>

**Week 3 part 1 - creating dataframe**

In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 25.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
from bs4 import BeautifulSoup
import requests  
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib") 
tables = soup.find_all('table')

In [4]:
Toronto_data = pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood"])


In [5]:
for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        if (col[1].text.strip() != 'Not assigned'):
            pcode = col[0].text.strip()
            bor = col[1].text.strip()
            neigh = col[2].text.strip()
            Toronto_data = Toronto_data.append({"Postal Code":pcode, "Borough":bor, "Neighbourhood":neigh}, ignore_index=True)

Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
Toronto_data.shape

(103, 3)

**Week 3 part 2 - adding coordinates**

In [7]:
!pip install pgeocode

In [8]:
import pgeocode
nomi = pgeocode.Nominatim('CA')

In [9]:
T_data =  pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood","Latitude","Longitude"])


In [10]:
for ind in Toronto_data.index:
    pcode = Toronto_data['Postal Code'][ind]
    bor = Toronto_data['Borough'][ind]
    neigh = Toronto_data['Neighbourhood'][ind]
    latt = nomi.query_postal_code(pcode).latitude
    long = nomi.query_postal_code(pcode).longitude
    
    T_data  = T_data.append({"Postal Code":pcode, "Borough":bor, "Neighbourhood":neigh, "Latitude":latt, "Longitude":long}, ignore_index=True)
    

In [21]:
T_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


**Week 3 part 3 - Explore and cluster the neighborhoods in Toronto.**

In [14]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarc

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
# delete NaN value
T_data = T_data.drop(labels=76, axis=0)

In [19]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.1.5               |   py36h284efc9_0        11.3 MB  conda-forge
    pytz-2021.1                |     pyhd8ed1ab_0         239 KB  conda-forge
    toolz-0.11.1               |             p

In [24]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [25]:
# create map of Toronto using latitude and longitude values
map_TORONTO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(T_data['Latitude'], T_data['Longitude'], T_data['Borough'], T_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TORONTO)  
    
map_TORONTO